In [1]:
###Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from IPython.display import Image
from IPython.core.display import HTML
import datetime
import math
import scipy.optimize as optimize
import statistics
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy import stats
import scipy
import warnings
from scipy.stats import norm
from sklearn import datasets, linear_model, cross_validation
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold 
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
from IPython.display import Image
from IPython.core.display import HTML 
import xgboost
from collections import defaultdict

C:\Users\James\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
### data loading 
df = pd.read_csv('./data/loan.csv', low_memory=False)

In [3]:
### data loading & cleaning
df = df.drop_duplicates(subset='id').reset_index(drop=True)
df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['id', 'funded_amnt','int_rate','installment','grade'])
df = df[np.isfinite(df['annual_inc']) & np.isfinite(df['int_rate']) &  np.isfinite(df['funded_amnt'])].copy()

In [4]:
###changing data type to appropriate date type 
df['last_pymnt_d']=pd.to_datetime(df.last_pymnt_d)
df['issue_d']=pd.to_datetime(df.issue_d)

In [5]:
###fill missing last_payments(no payments) with issue date, may want a separete column for this treatment
###This is for calculation of IRRs later
df['last_pymnt_d'].fillna(df.issue_d,inplace=True)

In [6]:
###Some pre-processing for ease of calculations
df['issue_yr'] = df.issue_d.dt.year
df['issue_mo']= df.issue_d.dt.month
df['last_pymnt_yr'] = df.last_pymnt_d.dt.year
df['last_pymnt_mo']= df.last_pymnt_d.dt.month
df['mo_diff'] = pd.to_numeric((df['last_pymnt_yr'] - 
                          df['issue_yr'])*12 + df['last_pymnt_mo'] -df['issue_mo'])

In [7]:
###Flag for completed loans
searchfor = ['Fully Paid', 'Charged Off', 'Default']
defaults = ['Charged Off', 'Default']
df['loan_completion_flag']=  np.where(df['loan_status'].str.contains('|'.join(searchfor)) ,1, np.nan)
###Flag for fully paid loans
df['fully_paid'] = np.where(df['loan_status'].str.contains('Fully Paid') ,1, 
                                  np.where(df['loan_status'].str.contains('|'.join(defaults)) ,0,np.nan))


In [8]:
###Average payment = Total payment - recoveries - last payment amount over the life -1 month of the investment
df['avg_pymnt'] = (df['total_pymnt']-df['recoveries']-df['last_pymnt_amnt'])/(np.maximum((df['mo_diff']-1),0))
###Treating infinites that appear when there is no payment or only 1 payment 
df['avg_pymnt'] = (df['avg_pymnt']).replace(np.Inf,0)
df['avg_pymnt'] = (df['avg_pymnt']).replace(-np.Inf,0)

In [9]:
###IRR calculations
###Input: a row of a dataframe with lending data 
def irr_calc(x):  
    ##varible initialization
    initial_invest = -x['funded_amnt']
    avg_payment = x['avg_pymnt']
    num_payments = np.max(int(x['mo_diff'])-1,0)
    recovery = x['recoveries'] -x['collection_recovery_fee']
    recovery_duration = np.maximum(36 - num_payments + 1 + 12,12)
    avg_recovery = recovery/recovery_duration
    last_payment_amount = x['last_pymnt_amnt']
    ###IRR calculation, input: series of cash flows, total payment - recoveries
    ###evenly divided and spread across the life of the loan and finally recovery and chargeoff fees
    return ((np.irr([initial_invest]+[avg_payment]*num_payments + [last_payment_amount] +
                    [avg_recovery]*recovery_duration)+1)**12-1)

In [10]:
###Calculating at a row level, individual security IRRs. Method will be faulty for loans that didn't mature.
###Warning: the calculation takes a fair amount of time ~few minutes
df['irr']=df.apply(irr_calc, axis=1)

In [11]:
##NaNs returned from IRRs with 0 payments should be -100% return 
df['irr']=df['irr'].replace(np.NaN,-1)

In [12]:
####Filter down to completed loans and has at least 36 months of possible history
df_filtered = df[df['loan_status'].str.contains('|'.join(searchfor))].query("term == ' 36 months' and issue_yr <=2012").copy()

In [13]:
##making grade flags
grade_flags = pd.get_dummies(df_filtered.grade) 
home_flag = pd.get_dummies(df_filtered.home_ownership) 
purpose_flag = pd.get_dummies(df_filtered.purpose)
df_filtered=pd.concat([df_filtered,grade_flags,home_flag,purpose_flag], axis=1)


In [14]:
def calc_train_error(X_train, y_train, model):
    '''returns in-sample error for already fit model.'''
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    return mse
    
def calc_validation_error(X_test, y_test, model):
    '''returns out-of-sample error for already fit model.'''
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    return mse
    
def calc_metrics(X_train, y_train, X_test, y_test, model):
    '''fits model and returns the RMSE for in-sample error and out-of-sample error'''
    model.fit(X_train, y_train)
    train_error = calc_train_error(X_train, y_train, model)
    validation_error = calc_validation_error(X_test, y_test, model)
    return train_error, validation_error

# Model Testing

In [15]:
columns= ["int_rate","annual_inc","funded_amnt","A","B","C","D","E","F","G","MORTGAGE","NONE",
          "OTHER","OWN","RENT","car","credit_card","debt_consolidation","educational","home_improvement","house",
         "major_purchase","medical","moving","other","renewable_energy","small_business","vacation","wedding"]

In the last [post](http://jameslee.posthaven.com/peer-to-peer-lending-markets-part-2-using-linear-regression-and-gradient-boosting-regression-to-outperform-the-market), gbm regression model and linear regression were pitted against random loan selection method to see if we can outperform a well diversified market portfolio, and we saw that our models dramatically outperformed the market portfolio. So with that exploratory work out of the way, I will aim to build a model with extensive tuning that can potentially be used in practice. We will split our data into train and test sets and further split the train set using k-fold cross validation to tune our parameters. Our final model will be compared vs other models using the test set that was held out.

For feature selection ,we will first feed in all plausible features, calculate their importances using GBM model's built-in feature importance function. Using this as a ranked order, we will deduct bottom 20-th percentile features out of our model, then see the model's performance using that many features across different validation sets using 5 fold cross-validation. 

Ideally, we tune the parameters and feature selection at the same time, but this is not feasible for my computation resources. So we will first feed all the features, get the right feature numbers, then tune our parameters using those number of features.

In [16]:
df_train, df_test = cross_validation.train_test_split(df_filtered,test_size=.3)

We first establish a standard set of parameters and train an initial model to calcualte feature importance across all the features on our training set. 

In [17]:
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(df_train[columns], df_train['irr'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [18]:
importance = clf.feature_importances_

We then remove 20% of the features at a time then see how the new models perform using k-fold cross validation. 

In [218]:
kf = KFold(n_splits=4) # Define the split - into 4
all_trials = []
for i in range(int(np.ceil(len(columns)/6))):
    sorted_features = importance.argsort()[-(len(importance)-i*6):][::-1]
    for index, (train, test) in enumerate(kf.split(df_train)):
        x_train = df_train.iloc[train][columns].iloc[:,sorted_features]
        y_train = df_train.iloc[train]['irr']
        y_test = df_train.iloc[test]['irr']
        x_test = df_train.iloc[test][columns].iloc[:,sorted_features]

        params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
              'learning_rate': 0.01, 'loss': 'ls'}

        clf = ensemble.GradientBoostingRegressor(**params)
        clf.fit(x_train, y_train)
        gbm_predictions = clf.predict(x_test)
        df_temp = pd.DataFrame({'irr':y_test, 'gbm_predictions':gbm_predictions})
        gbm_returns = np.mean(df_temp.nlargest(1000, 'gbm_predictions')['irr'])
        all_trials.append({
            'n_features': len(x_train.columns),
            'mse': calc_train_error(x_test,y_test,clf),
            'returns': gbm_returns
        })

In [219]:
output=pd.DataFrame.from_records(all_trials)

In [220]:
output.groupby(['n_features'])['mse','returns'].mean()

,mse,returns
n_features,,
5,0.045378,0.108145
11,0.045230,0.107996
17,0.045176,0.110360
23,0.045175,0.110418
29,0.045167,0.110371


Our cross validation yields the lowest error as well as the higest returns with n_features = 17. We will be using this as a basis for our hyperparameter tuning. 

We have our set of parameters that we will be using. Now it's time to tune our parameters and fit a model on our entire training set. 

Lower learning rate will always yield to bette result. But our constraint is computation time. So we can't set it too low

In [19]:
len(df_train)

52411

In [ ]:
parameters = {
    "loss":["ls"],
    "learning_rate": [0.05],
    "min_samples_split":[300],
    "min_samples_leaf": [50],
    "max_depth":[8],
    "max_features":["sqrt"],
    "subsample":[.8],
    "n_estimators":[25,50, 100, 200, 500, 1000]
    }
clf = GridSearchCV(ensemble.GradientBoostingRegressor(), parameters, cv=10, n_jobs=-1)

In [237]:
parameters = {
    "n_estimators":[25,50, 100, 200, 500, 1000]
    }
clf_ntrees = GridSearchCV(ensemble.GradientBoostingRegressor(learning_rate=0.05, min_samples_split=300,
                                                      min_samples_leaf=50,max_depth=8,max_features='sqrt',
                                                      subsample=0.8,random_state=10), parameters, cv=5, n_jobs=-1)

In [238]:
clf_ntrees.fit(df_train[columns].iloc[:,importance.argsort()[-17:][::-1]], df_train['irr'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=50, min_samples_split=300,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=10, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [25, 50, 100, 200, 500, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [241]:
print(clf_ntrees.grid_scores_, clf_ntrees.best_params_, clf_ntrees.best_score_)

[mean: 0.01574, std: 0.00071, params: {'n_estimators': 25}, mean: 0.01832, std: 0.00117, params: {'n_estimators': 50}, mean: 0.01881, std: 0.00170, params: {'n_estimators': 100}, mean: 0.01730, std: 0.00199, params: {'n_estimators': 200}, mean: 0.01091, std: 0.00266, params: {'n_estimators': 500}, mean: 0.00098, std: 0.00256, params: {'n_estimators': 1000}] {'n_estimators': 100} 0.0188129230061


In [246]:
 np.linspace(60, 150, 10).astype(int)

array([ 60,  70,  80,  90, 100, 110, 120, 130, 140, 150])

In [247]:
parameters = {
    "n_estimators": np.linspace(60, 150, 10).astype(int)
    }
clf_ntrees2 = GridSearchCV(ensemble.GradientBoostingRegressor(learning_rate=0.05, min_samples_split=300,
                                                      min_samples_leaf=50,max_depth=8,max_features='sqrt',
                                                      subsample=0.8,random_state=10), parameters, cv=5, n_jobs=-1)

In [248]:
clf_ntrees2.fit(df_train[columns].iloc[:,importance.argsort()[-17:][::-1]], df_train['irr'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=50, min_samples_split=300,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=10, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': array([ 60,  70,  80,  90, 100, 110, 120, 130, 140, 150])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [250]:
print( clf_ntrees2.best_params_)

{'n_estimators': 80}


80 is the best n_trees we will move onto other parameters. The order of tuning variables should be decided carefully. You should take the variables with a higher impact on outcome first. For instance, max_depth and min_samples_split have a significant impact and we’re tuning those first.

In [24]:
parameters2= {
    'max_depth':  np.linspace(4, 16, 7).astype(int), 'min_samples_split': np.linspace(200,1000, 5).astype(int)
}
clf_max_min = GridSearchCV(ensemble.GradientBoostingRegressor(learning_rate=0.05, 
                                                      min_samples_leaf=50,max_features='sqrt',
                                                      subsample=0.8,random_state=10, n_estimators = 80), parameters2, cv=5, n_jobs=-1)

In [25]:
clf_max_min.fit(df_train[columns].iloc[:,importance.argsort()[-17:][::-1]], df_train['irr'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=50, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=80, presort='auto',
             random_state=10, subsample=0.8, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': array([ 4,  6,  8, 10, 12, 14, 16]), 'min_samples_split': [200, 400, 600, 800, 987, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
print(clf_max_min.best_params_)

{'max_depth': 6, 'min_samples_split': 400}


In [32]:
np.linspace(30,80,6)

array([ 30.,  40.,  50.,  60.,  70.,  80.])

In [34]:
parameters3= {
    'max_depth':  np.linspace(5,7,3).astype(int), 'min_samples_split': np.linspace(300,500, 5).astype(int), 'min_samples_leaf': np.linspace(30,80,6).astype(int)
}
clf_max_min_leaves = GridSearchCV(ensemble.GradientBoostingRegressor(learning_rate=0.05, 
                                                      max_features='sqrt',
                                                      subsample=0.8,random_state=10, n_estimators = 80), parameters3, cv=5, n_jobs=-1)

In [35]:
clf_max_min_leaves.fit(df_train[columns].iloc[:,importance.argsort()[-17:][::-1]], df_train['irr'])

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=80, presort='auto',
             random_state=10, subsample=0.8, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': array([5, 6, 7]), 'min_samples_split': array([300, 350, 400, 450, 500]), 'min_samples_leaf': array([30, 40, 50, 60, 70, 80])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [38]:
clf_max_min_leaves.best_score_ , clf_max_min_leaves.best_params_

(0.018795155337357913,
 {'max_depth': 7, 'min_samples_leaf': 40, 'min_samples_split': 400})

In [21]:
np.linspace(1,17,17)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17.])

In [51]:
parameters4= {
    'max_features':  np.linspace(1,17,17).astype(int)
}
clf_max_min_leaves_feature = GridSearchCV(ensemble.GradientBoostingRegressor(learning_rate=0.05, 
                                                      max_depth = 7, min_samples_leaf = 40, min_samples_split = 400,
                                                      subsample=0.8,random_state=10, n_estimators = 80), parameters4, cv=5, n_jobs=-1)

In [ ]:
clf_max_min_leaves_feature.fit(df_train[columns].iloc[:,importance.argsort()[-17:][::-1]], df_train['irr'])

In [26]:
clf_max_min_leaves_feature.best_score_ , clf_max_min_leaves_feature.best_params_

(0.019032377455621013, {'max_features': 6})

In [19]:
parameters5= {
    'subsample':  np.linspace(.5,1,6)
}
clf_5 = GridSearchCV(ensemble.GradientBoostingRegressor(learning_rate=0.05, max_features=6,
                                                      max_depth = 7, min_samples_leaf = 40, min_samples_split = 400,
                                                      random_state=10, n_estimators = 80), parameters5, cv=5, n_jobs=-1)

In [71]:
clf_5.fit(df_train[columns].iloc[:,importance.argsort()[-17:][::-1]], df_train['irr'])
clf_5.best_score_ , clf_5.best_params_

(0.019032377455621013, {'subsample': 0.8})

In [ ]:
np.linspace(1000,4000,4).astype(int)

In [43]:
np.linspace(100,4000,11)

array([ 100.,  490.,  880., 1270., 1660., 2050., 2440., 2830., 3220.,
       3610., 4000.])

In [32]:
parameters6= {
    'n_estimators':  np.linspace(100,4000,11).astype(int)
}
clf_6= GridSearchCV(ensemble.GradientBoostingRegressor(learning_rate=0.01, max_features=6,
                                                      max_depth = 7, min_samples_leaf = 40, min_samples_split = 400,
                                                      random_state=10,subsample=.8), parameters6, cv=5, n_jobs=-1)

In [33]:
clf_6.fit(df_train[columns].iloc[:,importance.argsort()[-17:][::-1]], df_train['irr'])
clf_6.best_score_ , clf_6.best_params_

(0.01710893682071948, {'n_estimators': 80})